<a href="https://www.kaggle.com/code/m26102002/spotify-2025-global-music-market-analytics?scriptVersionId=285124211" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spotify-global-music-dataset-20092025/track_data_final.csv
/kaggle/input/spotify-global-music-dataset-20092025/spotify_data clean.csv


# 1. Data Preparation and Filtering

In [2]:
# a) load datset

df = pd.read_csv('/kaggle/input/spotify-global-music-dataset-20092025/spotify_data clean.csv')
df.head(2)


,track_id,track_name,track_number,track_popularity,explicit,artist_name,artist_popularity,artist_followers,artist_genres,album_id,album_name,album_release_date,album_total_tracks,album_type,track_duration_min
0,3EJS5LyekDim1Tf5rBFmZl,Trippy Mane (ft. Project Pat),4,0,True,Diplo,77,2812821,moombahton,5QRFnGnBeMGePBKF2xTz5z,"d00mscrvll, Vol. 1",2025-10-31,9,album,1.55
1,1oQW6G2ZiwMuHqlPpP27DB,OMG!,1,0,True,Yelawolf,64,2363438,"country hip hop, southern hip hop",4SUmmwnv0xTjRcLdjczGg2,OMG!,2025-10-31,1,single,3.07


In [3]:
print(df.shape)
print(df.info())

(8582, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8582 entries, 0 to 8581
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   track_id            8582 non-null   object 
 1   track_name          8582 non-null   object 
 2   track_number        8582 non-null   int64  
 3   track_popularity    8582 non-null   int64  
 4   explicit            8582 non-null   bool   
 5   artist_name         8579 non-null   object 
 6   artist_popularity   8582 non-null   int64  
 7   artist_followers    8582 non-null   int64  
 8   artist_genres       5221 non-null   object 
 9   album_id            8582 non-null   object 
 10  album_name          8582 non-null   object 
 11  album_release_date  8582 non-null   object 
 12  album_total_tracks  8582 non-null   int64  
 13  album_type          8582 non-null   object 
 14  track_duration_min  8582 non-null   float64
dtypes: bool(1), float64(1), int64(5), object(8)


In [4]:
# b) Date transformation and filtering

RELEASE_DATE_COL = 'album_release_date'
TARGET_YEAR = 2025

# convert release date to datetime, extracting the year for filtering

df[RELEASE_DATE_COL] = pd.to_datetime(df[RELEASE_DATE_COL], errors='coerce')
df['release_year'] = df[RELEASE_DATE_COL].dt.year

In [5]:
# filter for 2025 data only

df_2025 = df[df['release_year'] == TARGET_YEAR].copy()

In [6]:
# c) Handle Missing Values in core analysis columns

core_cols = ['track_popularity', 'track_duration_min', 'artist_name']
df_2025.dropna(subset=core_cols, inplace=True)

In [7]:
print(f"Total cleaned 2025 records used for analysis: {len(df_2025)}")

Total cleaned 2025 records used for analysis: 765


# Insight 1: The lenght of success ---

In [8]:
# a) define popularity tiers (using quartiles)

low_quantile = df_2025['track_popularity'].quantile(0.25)

In [9]:
high_quantile = df_2025['track_popularity'].quantile(0.75)

In [10]:
def popularity_tier(popularity):
    if popularity >= high_quantile:
        return 'A. Top Hits (Top 25%)'
    elif popularity >= low_quantile:
        return 'B. Mid-Range'
    else:
        return 'C. Low Popularity (Bottom 25%)'

df_2025['popularity_tier'] = df_2025['track_popularity'].apply(popularity_tier)

In [11]:
# b. Calculate the average duration for each tier

duration_analysis =  df_2025.groupby('popularity_tier')['track_duration_min'].mean().reset_index()

duration_analysis.columns = ['Popularity Tier', 'Average Duration (Minutes)']
duration_analysis.sort_values(by='Popularity Tier', inplace=True)

In [12]:
# Export the result to CSV
duration_analysis.to_csv('01_duration_vs_popularity_2025.csv', index=False)

In [13]:
print("\n--- Insight 1 Data (Duration vs. Popularity) Generated ---")
print(duration_analysis)


--- Insight 1 Data (Duration vs. Popularity) Generated ---
                  Popularity Tier  Average Duration (Minutes)
0           A. Top Hits (Top 25%)                    3.278164
1                    B. Mid-Range                    3.003237
2  C. Low Popularity (Bottom 25%)                    3.011629


# Insight 2 - The Dominant Sounds of 2025 (Genre Analysis) ---

In [14]:
# a) Clean and Prepare genre Data

df_2025['artist_genres'] = df_2025['artist_genres'].fillna('').astype(str).str.lower()
df_2025['artist_genres_cleaned'] = df_2025['artist_genres'].str.replace(r'\[|\]|\'', '', regex=True)

In [15]:
# Split by comma and explode (create one row per genre per track)

genre_exploded = df_2025.assign(genre=df_2025['artist_genres_cleaned'].str.split(',')).explode('genre')

In [16]:
genre_exploded['genre'] = genre_exploded['genre'].str.strip()

In [17]:
#  Remove empty/placeholder strings

genre_exploded = genre_exploded[genre_exploded['genre'].isin(['', 'n/a', 'nan']) == False].copy()

In [18]:
# b. find top 10 Most Frequent Genres

top_genres = genre_exploded['genre'].value_counts().nlargest(10).index.tolist()

In [19]:
# c. Calculate the average popularity for these top 10 genres

genre_popularity_analysis = (
    genre_exploded[genre_exploded['genre'].isin(top_genres)].groupby('genre')['track_popularity'].mean().sort_values(ascending = False).reset_index())


In [20]:
genre_popularity_analysis.to_csv('02_genre_popularity_2025.csv', index=False)

In [21]:
print("\n--- Insight 2 Data (Genre Popularity) Generated ---")
print(genre_popularity_analysis)


--- Insight 2 Data (Genre Popularity) Generated ---
             genre  track_popularity
0            k-pop         77.310345
1              pop         72.849057
2          country         70.055556
3          hip hop         65.240000
4          r&b pop         65.086957
5  alternative r&b         64.777778
6          art pop         64.750000
7       mahraganat         46.677419
8         dark r&b         46.322581
9     egyptian pop         46.312500


# Insights 3: Artist Investment Profile

In [22]:
# a) define artist profile tiers based on their global popularity

ARTIST_POP_CUTOFF = df_2025['artist_popularity'].quantile(0.75)

In [23]:
def artist_profile(pop_score):
    if pop_score >= ARTIST_POP_CUTOFF:
        return 'A. Gloabl Superstars (TOP25%)'
    else:
        return 'B. Mid-Tier / Newcomers'
df_2025['artist_profile'] = df_2025['artist_popularity'].apply(artist_profile)

In [24]:
# b. Calculated the avg TRACK popularity for each Artist Profile

artist_analysis = df_2025.groupby('artist_profile')['track_popularity'].mean().reset_index()

artist_analysis.columns = ['Artist Profile', 'Average Track Popularity (0-100)']
artist_analysis.sort_values(by='Artist Profile', inplace = True)

In [25]:
# Export the result to CSV

artist_analysis.to_csv('03_artist_investment_2025.csv', index=False)

print("\n--- Insight 3 Data (Artist Profile) Generated ---")
print(artist_analysis)


--- Insight 3 Data (Artist Profile) Generated ---
                  Artist Profile  Average Track Popularity (0-100)
0  A. Gloabl Superstars (TOP25%)                         74.357513
1        B. Mid-Tier / Newcomers                         44.162587
